In [1]:
import numpy as np
from math import exp, log, sqrt, erf

def black_scholes_call(S0:float, K:float, r:float, sigma:float, T:float):
    """
    Calculates the analytical Black-Scholes price for a European call option.

    :param S0: Intial Stock Price at t=0
    :param K: Strike Price
    :param r: Risk Free Rate
    :param sigma: Volatility
    :param T: Time to Maturity

    :return: The exact Black-Scholes price of the European call option
    """

    # Handle edge cases where time or volatility is zero
    if T <= 0 or sigma <= 0:
        return np.maximum(S0 - K, 0.0) * exp(-r * T)

    # Standard Black-Scholes formula components
    d1 = (log(S0 / K) + (r + 0.5 * sigma**2) * T) / (sigma * sqrt(T))
    d2 = d1 - sigma * sqrt(T)

    # Calculate the cumulative distribution function (CDF) of the standard normal distribution.
    cdf_d1 = 0.5 * (1.0 + erf(d1 / sqrt(2.0)))
    cdf_d2 = 0.5 * (1.0 + erf(d2 / sqrt(2.0)))

    price = S0 * cdf_d1 - K * exp(-r * T) * cdf_d2
    return price

def european_call_mc(S0: float, K:float, r:float, sigma:float, T:float, num_paths):
    """
    Prices a European call option using a simple Monte Carlo simulation

    Parameters:
    :param S0: Initial stock price.
    :param K: Strike price.
    :param r: Risk-free interest rate.
    :param sigma: Volatility of the underlying asset.
    :param T: Time to maturity in years.
    :param num_paths: Total number of Monte Carlo simulation paths.

    Returns:
    tuple: (final_price, standard_error)

    """
    # Generate standard normal random variables
    Z = np.random.normal(0.0, 1.0, size=num_paths // 2)

    # Simulate the final stock price at maturity T
    drift_term = (r - 0.5 * sigma**2) * T
    diffusion_term = sigma * sqrt(T)

    # Simulate the standard paths
    S_T = S0 * np.exp(drift_term + diffusion_term * Z)

    # Calculate the payoff for each path
    payoff = np.maximum(S_T - K, 0)

    # Discount the average payoff and calculate statistics
    discounted_payoff = np.exp(-r * T) * payoff

    final_price = np.mean(discounted_payoff)
    standard_error = np.std(discounted_payoff, ddof=1) / np.sqrt(num_paths // 2)

    return final_price, standard_error

if __name__ == "__main__":
    S0 = 1
    K = 1
    r = 0.05
    sigma = 0.20
    T = 1.0

    # Total number of simulation paths
    num_paths = 1000000

    print("--- Standard Monte Carlo for European Option ---")
    print(f"Parameters: S0={S0}, K={K}, r={r}, sigma={sigma}, T={T}")
    print(f"Simulation: {num_paths} paths")

    # Calculate the price using Monte Carlo
    mc_price, std_error = european_call_mc(S0, K, r, sigma, T, num_paths)

    # Calculate the exact price using Black-Scholes for comparison
    bs_price = black_scholes_call(S0, K, r, sigma, T)

    # Calculate Absolute Error
    absolute_error = abs(mc_price - bs_price)

    # Calculate Normalised Error
    normalised_error = absolute_error / bs_price

    print(f"\nBlack-Scholes Exact Price: {bs_price:.6f}")
    print(f"Monte Carlo Estimated Price: {mc_price:.6f}")
    print(f"Standard Error:              {std_error:.6f}")
    print(f"Absolute Error:              {absolute_error:.6f}")
    print(f"Normalised Error:            {normalised_error:.6f}")

--- Standard Monte Carlo for European Option ---
Parameters: S0=1, K=1, r=0.05, sigma=0.2, T=1.0
Simulation: 1000000 paths

Black-Scholes Exact Price: 0.104506
Monte Carlo Estimated Price: 0.104265
Standard Error:              0.000208
Absolute Error:              0.000241
Normalized Error:            0.002307
